# Grabbing Directory Information

Here, we grab all of the information we can get out of the directory and stuff it into a pandas dataframe (and excel spreadsheet).

This parsing is ridiculously fragile, and built specifically around the data that I see on June 4, 2015

In [1]:
from __future__ import division
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np, scipy as sp, pandas as pd, matplotlib as mpl
from bs4 import BeautifulSoup

In [2]:
soup = BeautifulSoup(open('data/EC_Student_Directory.html'))
people = soup.find_all("div", class_="person".split(), limit=300)
print "I found {lp} people".format(lp=len(people))

I found 201 people


Well that's broken. Let's be simpleminded.

In [3]:
divs = []
thisdiv = ''
closecount = 0
inperson = False
for line in open('data/EC_Student_Directory.html'):
    if line.strip() == '<div class="person">':
        if thisdiv != '':
            raise Exception('parse error')
        thisdiv = line
        closecount = 0
        inperson = True
    if inperson:
        thisdiv += line
        if line.strip() == '</div>':
            closecount += 1
        if closecount == 3:
            divs.append(thisdiv)
            thisdiv, closecount, inperson = '', 0, False
print len(divs)

890


In [4]:
people = []
def person(div):
    d = {}
    for line in div.split('\n'):
        line = line.strip()
        if line.startswith('<a href='):
            d['Full Name'] = line.split('>')[1].split('<')[0]
        elif line.startswith('<p class="academic'):
            line = line + '<phonyend>'
            for marker in ('Preferred Name','Class Year','Major(s)','Minor(s)','Advisor'):
                if marker in line:
                    try:
                        d[marker] = line.split('{m}: '.format(m=marker))[1].split('<')[0]
                    except IndexError:
                        d[marker] = ''
        elif line.startswith('<p class="contact'):
            d['email'] = line.split('mailto:')[1].split('"')[0]
        if d:
            d['Last Name'] = d['Full Name'].split()[-1]
            d['First Name'] = d['Full Name'].split()[0]

    return d
for div in divs:
    people.append(person(div))

In [5]:
df = pd.DataFrame(columns = people[0].keys(), index=range(1,len(people)+1))
for (i,person) in enumerate(people):
    df.loc[i+1] = pd.Series(person)

In [6]:
df.fillna('').to_excel('currentstudents.xlsx',sheet_name='Sheet 1')

In [7]:
#advising = df['Advisor'].value_counts()

In [8]:
#http://stackoverflow.com/questions/19124601/is-there-a-way-to-pretty-print-the-entire-pandas-series-dataframe
def print_full(x):
    pd.set_option('display.max_rows', len(x))
    print(x)
    pd.reset_option('display.max_rows')
#print_full(advising)